In [26]:
import os
import albumentations as A
import cv2
import numpy as np
import random

In [27]:
# Define the root folder containing category folders
root_folder = r"D:\Python_projects\UNet-Document-Layout-Analysis-Research\data\project-7-at-2024-11-28-20-06-811bd479"

# List of categories in the root folder
categories = os.listdir(root_folder)

In [28]:
# Seed for reproducibility
random.seed(42)

In [29]:
def albumentation_pipeline(image, label):
    """
    Apply augmentation pipeline to image and label pair.
    """
    # Define augmentations
    transform = A.Compose([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        A.GaussianBlur(blur_limit=(3, 7), p=0.3),
        A.ToGray(p=0.3),  # Convert some images to grayscale
        A.Rotate(limit=3, p=0.5),
    ])

    # Apply augmentations
    augmented = transform(image=image, mask=label)
    return augmented['image'], augmented['mask']

In [30]:
# Preprocess all categories
for category in categories:
    # Combine root path with category path
    category_path = os.path.join(root_folder, category)
    image_folder = os.path.join(category_path, "Images")
    label_folder = os.path.join(category_path, "Labels")

    # Paths for augmented images and labels
    augmented_image_folder = os.path.join(category_path, "Augmented_Images")
    augmented_label_folder = os.path.join(category_path, "Augmented_Labels")
    os.makedirs(augmented_image_folder, exist_ok=True)
    os.makedirs(augmented_label_folder, exist_ok=True)

    # Read image and label files
    image_files = sorted(os.listdir(image_folder))
    label_files = sorted(os.listdir(label_folder))

    processed_files = set()

    for img_file, lbl_file in zip(image_files, label_files):
        # Skip duplicates
        if img_file in processed_files:
            continue
        processed_files.add(img_file)

        # Read image and label
        img_path = os.path.join(image_folder, img_file)
        lbl_path = os.path.join(label_folder, lbl_file)

        image = cv2.imread(img_path)
        label = cv2.imread(lbl_path, cv2.IMREAD_GRAYSCALE)  # Labels are single-channel

        # Ensure the dimensions of image and label match
        if image is None or label is None:
            print(f"Skipping files: {img_file}, {lbl_file} due to read errors.")
            continue

        if image.shape[:2] != label.shape[:2]:
            print(f"Resizing label for {lbl_file} to match {img_file}.")
            label = cv2.resize(label, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Apply albumentation pipeline
        for i in range(3):  # Generate multiple augmentations for each image
            augmented_image, augmented_label = albumentation_pipeline(image, label)

            # Save the augmented image and label
            augmented_image_path = os.path.join(augmented_image_folder, f"{i}_{img_file}")
            augmented_label_path = os.path.join(augmented_label_folder, f"{i}_{lbl_file}")

            cv2.imwrite(augmented_image_path, augmented_image)
            cv2.imwrite(augmented_label_path, augmented_label)

print("Preprocessing and augmentation completed successfully!")



Resizing label for task-36-annotation-12-by-1-tag-Author-0.png to match Doc_430.jpg.
Resizing label for task-460-annotation-438-by-1-tag-Author-0.png to match Doc_446.jpg.
Resizing label for task-476-annotation-454-by-1-tag-Author-0.png to match Doc_596.jpg.
Resizing label for task-626-annotation-604-by-1-tag-Author-0.png to match Doc_6.jpg.
Resizing label for task-104-annotation-80-by-1-tag-Caption-0.png to match Doc_116.jpg.
Resizing label for task-106-annotation-82-by-1-tag-Caption-0.png to match Doc_117.jpg.
Resizing label for task-107-annotation-83-by-1-tag-Caption-0.png to match Doc_118.jpg.
Resizing label for task-109-annotation-85-by-1-tag-Caption-0.png to match Doc_12.jpg.
Resizing label for task-110-annotation-86-by-1-tag-Caption-0.png to match Doc_15.jpg.
Resizing label for task-146-annotation-124-by-1-tag-Caption-0.png to match Doc_16.jpg.
Resizing label for task-147-annotation-125-by-1-tag-Caption-0.png to match Doc_165.jpg.
Resizing label for task-148-annotation-126-by-1-